# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

In [3]:
import numpy as np
import pandas as pd

BASE_DIR = r'C:\Users\Alex\Desktop\Academic\UChicago\FINM 37500\finm-fiderivs-2024'
import sys, os
sys.path.append(BASE_DIR)

from cmds.binomial import *
from cmds.ratecurves import *
from cmds.volskew import *
from cmds.ficcvol import *
from cmds.fi_binomial_model import FIBinomialModel

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

In [5]:
cap_df = pd.read_excel(r'../data/cap_quotes_2024-02-20.xlsx')
cap_df.head(5)

,date,USCNSQ1 SMKO Curncy,USCNSQ2 SMKO Curncy,USCNSQ3 SMKO Curncy,USCNSQ4 SMKO Curncy,USCNSQ5 SMKO Curncy,USCNSQ6 SMKO Curncy,USCNSQ7 SMKO Curncy,USCNSQ8 SMKO Curncy,USCNSQ9 SMKO Curncy,USCNSQ10 SMKO Curncy
0,maturity,1.002053,2.001369,3.000684,4.0,5.002053,6.001369,7.000684,8.0,9.002053,10.001369
1,2022-03-17 00:00:00,127.300000,108.500000,109.900000,108.5,107.300000,104.300000,101.000000,97.7,95.200000,93.200000
2,2022-03-18 00:00:00,96.200000,104.900000,108.000000,108.5,108.000000,105.000000,101.800000,98.3,95.400000,93.400000
3,2022-03-21 00:00:00,96.200000,105.000000,108.100000,108.5,107.900000,105.000000,101.700000,98.2,95.400000,93.400000
4,2022-03-22 00:00:00,75.100000,108.300000,115.200000,115.3,113.600000,109.900000,106.300000,102.8,99.900000,97.500000


In [6]:
swap_vol_df = pd.read_excel(r'../data/swaption_vol_data.xlsx')
swap_vol_df

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

In [ ]:
calc_fwdswaprate(cap_curves['discounts'], start, end, freqswap=4)


## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

In [7]:
cap_curves_df = pd.read_excel(r'../data/cap_curves_2024-02-20.xlsx')
cap_curves_df.head(5)

,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
1,0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
2,0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
3,1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
4,1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247


## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

***